In [ ]:
import yaml
import pandas as pd
import glob


In [ ]:
%cd ..

# Config

In [1]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

config

FileNotFoundError: [Errno 2] No such file or directory: 'params.yaml'

# Load Dataset


In [ ]:
# Get CSV files list of cosmetic shop dataset
path = config['data']['cosmetic_shop']
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame. This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
all_events   = pd.concat(df_list, ignore_index=True)


In [ ]:
all_events.shape

In [ ]:
all_events.sample(10)

# EDA
## Some Counts


In [ ]:
# Number of unique users
users_count = len(pd.unique(all_events['user_id']))
print("Unique users count: ", users_count)

# Number of unique products
products_count = len(pd.unique(all_events['product_id']))
print("Unique products count: ", products_count)